# Using the `kbresults` workflow
This notebook illustrates the normal workflow for `kbresults` functions. The first three functions modify files and directory structure and are typically executed from the command line. They're outlined in the notebook here as an example. In practice, `experiment_dir` is specified by the user and will remain unchanged throughout the workflow. Because the workflow modifies files and directories in-place, this notebook changes `experiment_dir` at each step to allow you to see how the operations proceed. The actions are destructive to the original directory structure so once they are executed you would have to reclone the repo to observe them in action again. 

The principal conventions for the workflow are that images are in TIFF format, housed within a `tiff` subdirectory within the parent `experiment_dir` directory, and are named with a unique alphanumeric character as the first digit and an integer as the second digit. The workflow is designed with the convention that the first digit encodes an experimental condition, and the second digit specifies the image/field of view. If more than one axon has been identified within a single image, each of those will eventually generate kymograph files where the final digit will be an integer that identifies that axon. 

The sample dataset contains four experimental conditions (a-d), representing a genotype and a drug applied in a balanced manner. There are different fields of view for each condition, and some fields have more than one axon for subsequent processing. 

In [3]:
from collections import OrderedDict

In [4]:
import kbresults as kb

## I. Prepare imaging files for kymograph generation
Set up `experiment_dir` to be a folder where everything starts and will be stored. Move all the original TIFF files into a subdirectory: `experiment_dir/tiff`. Then `filetiffs` will generate folders accordingly. 

In [ ]:
experiment_dir = 'filetiffs_example/'
kb.filetiffs('experiment_dir')

## II. Generate kymographs and repackage for KymoButler
After using `filetiffs`, the TIFF files will be in subfolders, which are opened in ImageJ to generate kymographs using KymoClear. To rearrange the resulting kymographs for processing with KymoButler, apply the `sortkymos` function. Then generate a ZIP of that directory for upload. 

In [ ]:
experiment_dir = 'sortkymos_example'
kb.sortkymos('experiment_dir')

## III. Rearrange KymoButler output
The `sortkymos` function generates a `4KymoButler` directory. Before proceeding, ensure that directory is intact and within `experiment_dir`, and contains ONLY the kymograph files generated in the preceeding steps. Those filenames will be used to organize all the data in the subsequent steps. 

Unzip the KymoButler output file to a directory `kboutput` (this directory name can be modified by the user), then run the `sortkbcsv` function. 

In [ ]:
experiment_dir = 'sortkbcsv_example'
kb.sortkbcsv('experiment_dir', 'kboutput')

## IV. Working with the final data
The final output from `sortkbcsv` is a `Results` directory with subdirectories for each measurement: frame to frame velocity, vesicle coordinates, etc. Each directory will contain CSV files where each represents the results for a particular axon, named according to the conventions described above. These CSVs can be further mined by the user. The `summaryframe` function is one such example: it will use the "batch summary" files to generate a Pandas dataframe for all the vesicles in an experiment, where each vesicle will have its experimental condition, field of view, axon number, average velocity, intensity, pauses, etc. listed in rows. Note that the "axon number" will count upward from 1 for each file containing the same starting character; the number will not reset between different fields of view. It also generates a separate dataframe for flux, with each row containing the results for a single axon. 

The new inputs for this function are two dictionaries (`dict_a`, `dict_b`) that translate the initial character into the different conditions within the experiment. Recall that this experiment has two independent variables (genotype and treatment) for a total of four conditions. This function depends solely on the `Results` directory generated in the previous step. 

This function also contains a `culture` flag, which allows the user to tag the entire experiment with an identifier. In this manner the dataframes can be concatenated with dataframes from other experiments to generate a larger set of results while still being able to partition the dataset by experimental cohort. 

In [1]:
experiment_dir = 'summaryframe_example'
genotype_dict = {'a':'wt', 'b':'wt', 'c':'ko', 'd':'ko'} 
treatment_dict = {'a':'control', 'b':'drug', 'c':'control', 'd':'drug'}

In [ ]:
df, flux = kb.summaryframe('summaryframe_example', genotype_dict, treatment_dict, culture='example')

In [ ]:
df.head()

In [ ]:
flux.head()